In [18]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

In [19]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [20]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [36]:
dataset = []
for i in range(0,len(data),3):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    dataset.append(subliste)

    


In [42]:
len(dataset)

617

In [43]:
serie1 = dataset[0]
print(serie1)
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'


csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'


for i in range(2):
    serie_object = Series(serie1[i])
    serie_data = serie_object.get_series_details()
    modality = serie_data['series']['Modality'] #CT ou PT
    print(modality)
    study_uid = serie_data['study']['StudyInstanceUID']
    filename = study_uid+'_'+'nifti_'+modality+'.nii'
    serie_object.get_numpy_array()
    serie_object.export_nifti(os.path.join(nifti_directory, filename))


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167', 'con_da_nov 7_2013.csv']
CT
["1.0", "0.0", "0.0", "0.0", "1.0", "0.0"]
PT
["1.0", "0.0", "0.0", "0.0", "1.0", "0.0"]


In [ ]:

nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'
csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'



for serie in dataset : 
    #nifti CT PT
    for i in range(2):
        serie_object = Series(serie[i]) #objet série 
        serie_data = serie_object.get_series_details()
        modality = serie_data['series']['Modality']
        study_uid = serie_data['study']['StudyInstanceUID']
        filename = study_uid+'_'+'nifti_'+modality+'.nii'
        serie_object.get_numpy_array()
        serie_object.export_nifti(os.path.join(nifti_directory, filename))

    #MaskBuilder
    mask = Mask_Builder(os.path.join(csv_directory, serie[2]), size_matrix)
    if is_correct_suv('nifti_array') #PT ou CT 

    

    #export en nifti 




        

In [35]:
None

In [ ]:
data[0]

In [21]:
csv = data[2::3]


In [22]:
serie_one = data[0::3] 
serie_two = data[1::3]
number_serie = len(serie_one)
all_serie = []
for i in range(number_serie):
    serie = []
    serie.append(serie_one[i])
    serie.append(serie_two[i])
    all_serie.append(serie)


In [39]:
None

In [40]:
None

In [32]:
None